# Lab 7: 提示词工程(Prompt Engineering 101)

# 1.chatGPT是如何实现的

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()  # 敏感信息不能明文展示
api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI

client = OpenAI(
            api_key=api_key,
            base_url="https://api.aaaapi.com/v1"
        )
# 创建一个简单的聊天完成请求
user_prompt = "你好,请介绍一下你自己"

response = client.chat.completions.create(
    model="gpt-4o",  
    messages=[
        {"role": "user", "content": user_prompt}
    ]
)
# 返回助手的回复
assistant_reply = response.choices[0].message.content


# 测试代码
print(f"用户: {user_prompt}")
print(f"AI助手: {assistant_reply}")


增加系统提示词

In [ ]:
system_prompt = "你是一个谜语人，总是用谜语来对话。谜底必须对应用户的答案"
response = client.chat.completions.create(
    model="gpt-4o",  
    messages=[
        {"role": "system", "content": system_prompt},  # 系统提示词是第一个加载的
        {"role": "user", "content": user_prompt}
    ]
)
# 返回助手的回复
assistant_reply = response.choices[0].message.content


# 测试代码
print(f"用户: {user_prompt}")
print(f"AI助手: {assistant_reply}")

增加对话上下文

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",  
    messages=[
        {"role": "system", "content": system_prompt},  # 系统提示词是第一个加载的
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "我是一本无字之书，读来似懂非懂。无需打开，便在手中展现玄机。这书名是啥？"},
        {"role": "user", "content": "难道你是灯泡？"}
    ]
)

print(f"用户: 难道你是天书？")
print(f"AI助手: {response.choices[0].message.content}")

提问，当我们说Prompt Engineering时，我们说的是什么？
- A: System Prompt
- B: 1st User Prompt
- C: All User Prompt
- D: Chat History?

# 2. 提高大模型输出质量的四种方式

## 2.1 Copilot

充分发挥大模型横向联想的能力。LLM eval其实很少做One attempt。
大模型的Token是极其便宜的，联想上的性价比爆击人类

In [38]:
def get_llm_response(user_prompt:str, model_name:str="gpt-4o"):
    response = client.chat.completions.create(
        model=model_name,  
        messages=[
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [39]:
def get_llm_response_with_token(user_prompt:str, model_name:str="gpt-4o"):
    response = client.chat.completions.create(
        model=model_name,  
        messages=[
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content, response.usage.prompt_tokens, response.usage.completion_tokens

In [ ]:
res, token_input, token_output = get_llm_response_with_token("请写3首藏头诗。主题是'昨天风好大'")
print(f"Token input: {token_input}, Token output: {token_output}, Cost is {token_input * 10/1000000*7.26 + token_output * 30/1000000*7.26} RMB")
display(print(res))


In [ ]:
display(print(get_llm_response("请写3首藏头诗。主题是'昨天风好大'", model_name="claude-3-5-sonnet-20240620")))

In [ ]:
display(print(get_llm_response("请写3首藏头诗。主题是'昨天风好大'", model_name="moonshot-v1-8k")))  # KIMI

## 2.2 In Context Learning

给大模型提供例子是让大模型“懂你”最快的方式

In [ ]:
context_prompt_1 = """请给我写一个拜年微信消息，模仿
```
我决定再也不发疯狂星期四文案了，疯狂星期四是五毒之首，是洪水猛兽，是离间我和朋友感情的元凶，是信任消失的的罪魁祸首，是纯情少年的无情杀手，疯狂星期四千万碰不得，疯狂星期四万万摸不得，每周四发疯四文案只能坏了大事，同意我的v我50作为封口费。
```
"""
display(print(get_llm_response(context_prompt_1)))

In [ ]:
context_prompt_2 = """请给我写一个拜年微信消息，模仿
```
To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. 
```
"""
display(print(get_llm_response(context_prompt_2)))

# 2.3 Chain of Thought

In [ ]:
prompt = "9.9和9.11哪个大"
display(print(get_llm_response(prompt)))

In [ ]:
prompt = "9.9和9.11哪个大。 Please think step by step."
display(print(get_llm_response(prompt)))

# 2.4 提示词模版

事实上提示词模版并不能保证提高大模型输出质量。在GPT 3.5年代，高度依赖提示词的结构化；在GPT 4年代，提示词模版的性价比降低了很多。

但是如果你要做一款大模型产品，提示词模版依然是必须掌握的技能。因为很有可能你的场景无法进行多轮对话，或者你不能指望你用户的提示词水平

### 三元素提示词模版

作为<角色>，你的目标是<目标>,你的任务是<任务>。

In [ ]:
prompt = "作为一个数学家，你的目标是仔细推理，确保答案正确。你的任务是比较9.9和9.11哪个大。"
display(print(get_llm_response(prompt)))

### COSTAR - 话术用
```
# 背景（Context）
# 目标（Objective）
# 风格（Style）
# 调性（Tone）
# 受众（Audience）
# 格式（Response）
```


In [ ]:
costar_prompt = """# 背景（Context）
教师节来临，要祝福教师同事节日快乐
# 目标（Objective）
1. 表扬教师爱岗敬业的精神
2. 激发荣誉感以鼓励其继续为学生服务
# 风格（Style）
婉约派诗词，类似柳永
# 调性（Tone）
温柔含蓄
# 受众（Audience）
25-35岁的高知女性为主
# 格式（Response）
词牌名：雨霖铃"""

display(print(get_llm_response(costar_prompt)))

### KIMI - 推理用
```
# 角色（Role）
# 背景（Background）
# 画像（Profile）
# 技能（Skills）
# 目标（Goals）
# 限制（Constraints）
# 工作流程（Workflow）
# 例子（Examples）
# 输出格式（Output Format）
```


# Now You Try!

阅读下面的材料，根据要求写作。

    随着互联网的普及、人工智能的应用，越来越多的问题能很快得到答案。那么，我们的问题是否会越来越少？

以上材料引发了你怎样的联想和思考？请写一篇文章。
要求：选准角度，确定立意，明确文体，自拟标题；不要套作，不得抄袭；不得泄露个人信息